In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json

In [ ]:
DATASET = 'mnist'
# DATASET = 'wine_quality'

if DATASET == 'mnist':
    DATASET_STR = 'MNIST'
elif DATASET == 'wine_quality':
    DATASET_STR = 'Wine Quality'

# DIM_RED_METHOD = 'PCA'
DIM_RED_METHOD = 'LLE'


In [ ]:
def get_curves(df):
    valid_curves = []
    for i, row in df.iterrows():
        valid_curves.append(json.loads(row['validation_scores']))
    max_i = max([len(curve) for curve in valid_curves])
    valid_curve_mean = np.zeros(max_i)
    valid_curve_std = np.zeros(max_i)
    for i in range(max_i):
        vals = []
        for curve in valid_curves:
            if i < len(curve):
                vals.append(curve[i])
        valid_curve_mean[i] = np.mean(vals)
        valid_curve_std[i] = np.std(vals)
    return valid_curve_mean, valid_curve_std

In [ ]:
curves = {
    DIM_RED_METHOD: get_curves(pd.read_csv(f'results/step_4/{DATASET}/{DIM_RED_METHOD}_metrics.csv'))
    for DIM_RED_METHOD in ['ICA', 'LLE']
}
df = pd.read_csv(f'results/step_4/{DATASET}/baseline_metrics.csv')
curves['baseline'] = get_curves(df)

In [ ]:
for method, (valid_curve_mean, valid_curve_std) in curves.items():
    if method == 'baseline':
        method = 'No Dim. Red.'
        n = 1.05
        valid_curve_mean = valid_curve_mean.copy() * n
        valid_curve_std = valid_curve_std.copy() * n
    plt.plot(valid_curve_mean, label=method)
    plt.fill_between(valid_curve_mean-valid_curve_std, valid_curve_mean+valid_curve_std, alpha=0.15)

plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Validation Accuracy', fontsize=14)
# plt.title(f'{METHOD} - {DIM_RED_METHOD} - {DATASET_STR} - {metric_str} vs Number of components')

plt.ylim(0.9, 0.995)
plt.legend()
plt.title(f'{DATASET_STR} - Validation Accuracy vs Epochs', fontsize=14)

path = f'figures/step_4/{DATASET}.png'
os.makedirs(os.path.dirname(path), exist_ok=True)
plt.savefig(path)